# BERT Review Votes

## Setup

In [2]:
!pip install transformers

     |████████████████████████████████| 3.8 MB 5.3 MB/s 
     |████████████████████████████████| 67 kB 5.1 MB/s 
     |████████████████████████████████| 895 kB 40.3 MB/s 
     |████████████████████████████████| 6.5 MB 30.8 MB/s 
     |████████████████████████████████| 596 kB 44.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
from google.colab import drive
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split
from transformers import DistilBertTokenizerFast
from transformers import TFDistilBertForSequenceClassification
import numpy as np
import pandas as pd
import tensorflow as tf

In [4]:
tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.122.200.10:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.122.200.10:8470


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [5]:
PATH_GDRIVE = '/content/drive'
drive.mount(PATH_GDRIVE, force_remount=True)

Mounted at /content/drive


In [6]:
# paths
DIR = 'drive/MyDrive/MSc Dissertation/data/'
PATH_SAMPLES = DIR + 'samples/review_polarity/%s.csv'
PATH_MODELS = DIR + 'models/review_playtime/%s_%d'
PATH_RESULTS = DIR + 'results/review_playtime/%s_%d'
# bert models
BERT_MODEL_ENG = 'distilbert-base-uncased'
BERT_MODEL_NENG = 'distilbert-base-multilingual-cased'

## Data Reading and Writing

In [7]:
def read_data(sample_name, train_size=0.8, test_size=0.5, seed=None):
    df = pd.read_csv(PATH_SAMPLES % sample_name)[['pt_review', 'text']].reset_index(drop=True)
    # scale votes
    scaler = RobustScaler()
    df['pt_review_scaled'] = scaler.fit_transform(np.array(df['pt_review']).reshape(-1, 1))
    # split data
    train, remaining = train_test_split(df, train_size=train_size, random_state=seed)
    validation, test = train_test_split(remaining, test_size=test_size, random_state=seed)
    return train, validation, test

In [8]:
def write_data_train(sample_name, run_num, train_history):
    with open((PATH_RESULTS % (sample_name, run_num)) + '_train.txt', 'w+') as f:
        f.write(str(train_history.params) + '\n')
        f.write(str(train_history.history) + '\n')

In [9]:
def write_data_test(sample_name, run_num, Y_test, Y_pred):
    with open((PATH_RESULTS % (sample_name, run_num)) + '_test.txt', 'w+') as f:
        f.write(str(Y_test) + '\n')
        f.write(str(Y_pred) + '\n')

## Model Training and Testing


In [10]:
def train_model(sample_name, train_data, validation_data, run_num=0, num_epochs=2, batch_size=16, learning_rate=5e-5, seed=None):
    with tpu_strategy.scope():
        # load pre-trained stuff
        if sample_name.startswith('eng'): model_name = BERT_MODEL_ENG
        else: model_name = BERT_MODEL_NENG
        tokenizer = DistilBertTokenizerFast.from_pretrained(model_name)
        classifier = TFDistilBertForSequenceClassification.from_pretrained(model_name, num_labels=1)
        # separate data
        Xt, Yt = train_data['text'].tolist(), train_data['pt_review_scaled'].tolist()
        Xv, Yv = validation_data['text'].tolist(), validation_data['pt_review_scaled'].tolist()
        # encode and format data
        encode = lambda x: tokenizer(x, truncation=True, padding=True, return_tensors='tf')
        format = lambda x, y: tf.data.Dataset.from_tensor_slices((dict(x), y)).shuffle(100, seed=seed)
        train_data = format(encode(Xt), Yt)
        validation_data = format(encode(Xv), Yv)
        # train and save the model
        optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, epsilon=1e-8)
        loss = tf.keras.losses.MeanSquaredError()
        classifier.compile(optimizer=optimizer, loss=loss)
    fit_results = classifier.fit(
        train_data.batch(batch_size),
        epochs=num_epochs,
        batch_size=batch_size,
        validation_data=validation_data.batch(batch_size)
    )
    classifier.save_pretrained(PATH_MODELS % (sample_name, run_num))
    write_data_train(sample_name, run_num, fit_results)

In [11]:
def test_model(sample_name, test_data, run_num=0, batch_size=16, seed=None):
    with tpu_strategy.scope():
        # load pre-trained stuff
        if sample_name.startswith('eng'): model_name = BERT_MODEL_ENG
        else: model_name = BERT_MODEL_NENG
        tokenizer = DistilBertTokenizerFast.from_pretrained(model_name)
        classifier = TFDistilBertForSequenceClassification.from_pretrained(PATH_MODELS % (sample_name, run_num), num_labels=1)
        # separate data
        Xt, Yt = test_data['text'].tolist(), test_data['pt_review_scaled'].tolist()
        #Yt_raw = test_data['votes_up'].tolist()
        # encode, format and convert data
        encode = lambda x: tokenizer(x, truncation=True, padding=True, return_tensors='tf')
        format = lambda x: tf.data.Dataset.from_tensor_slices(dict(x))
        convert = lambda y: y.logits[:,0][::8].tolist()
        Xt = format(encode(Xt))
        Yp = convert(classifier.predict(Xt))
    write_data_test(sample_name, run_num, Yt, Yp)

## Running Everything

In [12]:
batch_sizes = [16, 32, 64]
num_epochs = [2, 3, 4]
run_num = 0
sample_name = 'eng_any_any_100000'
seed = 1337
train, validation, test = read_data(sample_name, seed=seed)

In [ ]:
for batch_size in batch_sizes:
    for num_epoch in num_epochs:
        print(f'=== RUN_NUM={run_num} ===')
        train_model(sample_name, train, validation, run_num=run_num, seed=seed, batch_size=batch_size, num_epochs=num_epoch)
        test_model(sample_name, test, run_num=run_num, seed=seed, batch_size=batch_size)
        run_num += 1

In [13]:
sample_names = [
    'eng_any_any_100000', 'eng_any_long_100000', 'eng_any_short_100000',
    'any_any_any_100000', 'any_any_long_100000', 'any_any_short_100000',
]
seed=1337

In [ ]:
for sample_name in sample_names:
    train, validation, test = read_data(sample_name, seed=seed)
    train_model(sample_name, train, validation, run_num=10, batch_size=64, seed=seed)

In [ ]:
for sample_name in sample_names:
    _, _, test = read_data(sample_name, seed=seed)
    test_model(sample_name, test, run_num=10, batch_size=64, seed=seed)